In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

In [3]:
cd /content/drive/My\ Drive/Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [4]:
ls

ColabDriveGithub.ipynb  DataScienceMatrix/  Dzien4_meta.ipynb  Untitled0.ipynb


In [5]:
cd DataScienceMatrix/

/content/drive/My Drive/Colab Notebooks/DataScienceMatrix


In [6]:
ls

Data/  HelloGithub.ipynb  LICENSE  Matrix_one/  README.md


In [7]:
ls Data

men_shoes.csv


In [10]:
df = pd.read_csv("Data/men_shoes.csv",low_memory=False)
df.shape

(18280, 48)

In [0]:
def run_model(feats,model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  Y = df[ 'prices_amountmin'].values

  scores = cross_val_score(model,X,Y,scoring='neg_mean_absolute_error')
  return np.mean(scores),np.std(scores)

In [19]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [21]:
model = RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [20]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [22]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x)== 'nan' : return output_dict
  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [34]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [35]:
def get_name_feat(key):
  return 'feat_'+key
for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [36]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_nike id', 'feat_jewelry setting', 'feat_item character',
       'feat_theme', 'feat_auto', 'feat_wheel type',
       'feat_number of compartments', 'feat_age segment', 'feat_measurements:',
       'feat_color family'],
      dtype='object', length=526)

In [38]:
df[df['feat_athlete'].isnull()].shape

(18272, 526)

In [39]:
df.shape[0]

18280

In [0]:
keys_stats = {}
for key in keys:
  keys_stats[key] = df[False == df[get_name_feat(key)].isnull()].shape[0]/df.shape[0]*100

In [45]:
{k:v for k,v in keys_stats.items() if v>30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_color_cat'] = df['feat_color'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_material_cat'] = df['feat_material'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_sport_cat'] = df['feat_sport'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_style_cat'] = df['feat_style'].map(lambda x: str(x).lower()).factorize()[0]

for key in keys:
  df[get_name_feat(key)+'_cat'] = df[get_name_feat(key)].map(lambda x: str(x).lower()).factorize()[0]


In [67]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df [df.brand == df.feat_brand][['brand','feat_brand']].head()
df [df.brand == df.feat_brand].shape

(8846, 533)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]

In [82]:
feats = ['brand_cat','feat_brand_cat','feat_gender_cat','feat_material_cat','feat_metal type_cat','feat_style_cat','feat_shape_cat','feat_sport_cat']
#feats += feats_cat
#feats = list(set(feats))
model = RandomForestRegressor(max_depth=5,n_estimators=100)
run_model(feats,model)

(-57.29063655523627, 4.272945123404325)

In [83]:
X = df[ feats ].values
Y = df[ 'prices_amountmin'].values
m = RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
m.fit(X,Y)

perm = PermutationImportance(m,random_state=1).fit(X,Y)
eli5.show_weights(perm,feature_names = feats)


Weight,Feature
0.2575 ± 0.0073,brand_cat
0.1051 ± 0.0163,feat_material_cat
0.0248 ± 0.0041,feat_gender_cat
0.0178 ± 0.0007,feat_brand_cat
0.0128 ± 0.0004,feat_shape_cat
0.0087 ± 0.0003,feat_metal type_cat
0.0035 ± 0.0008,feat_style_cat
0.0001 ± 0.0000,feat_sport_cat


In [63]:
df['brand'].value_counts(normalize = True)

nike              0.097210
puma              0.033315
ralph lauren      0.028775
vans              0.021116
new balance       0.020295
                    ...   
protege           0.000055
dragonball z      0.000055
oliver sweeney    0.000055
primitive         0.000055
and 1             0.000055
Name: brand, Length: 1732, dtype: float64

In [70]:
df[df['brand'] =='nike'].features_parsed.sample(5).values

array([{'gender': 'boys', 'shoe category': 'boys shoes', 'color': 'black/metallic silver/chilling red', 'casual & dress shoe style': 'athletic', 'brand': 'nike', 'shoe width': 'medium'},
       {'primary color': 'blue', 'battery type': 'does not contain a battery', 'lens tint': 'grey', 'product in inches (l x w x h)': '6.0 x 2.0 x 2.0', 'assembled product dimensions (l x w x h)': '6.00 x 2.00 x 2.00 inches', 'is portable': 'y', 'frame material': 'nylon', 'model': 'ev0800-400', 'uv rating': '400', 'multi pack indicator': 'no', 'gender': 'men', 'brand': 'nike', 'age group': 'adult', 'lens type': 'sunglasses', 'features': 'wrap around fit; water repellent lenses; interchangeable lenses', 'material': 'plastic', 'model no.': 'ev0800-400', 'shape': 'semi-rimless', 'eyewear frame style': 'semi-rimless', 'shipping weight (in pounds)': '1.0', 'walmart no.': '554240966', 'manufacturer part number': 'ev0800-400'},
       {'gender': 'unisex', 'color': 'black/dark loden/volt', 'casual & dress shoe 

In [71]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
toddler                4
infant                 4
mens                   4
boys'                  3
women                  2
women ,�� unisex       2
youth                  2
men||women             2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64

In [81]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [84]:
ls

Data/  HelloGithub.ipynb  LICENSE  Matrix_one/  README.md


In [0]:
!git 